In [1]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN

In [7]:
dataset = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])

In [8]:
cleaned_dataset = dataset.dropna()

In [5]:
mlflow.set_experiment("EXP_6-Xgboost_hpt")

2025/09/24 12:27:07 INFO mlflow.tracking.fluent: Experiment with name 'EXP_6-Xgboost_hpt' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://apoorv-mlfow-bucket/754749025602314728', creation_time=1758697027208, experiment_id='754749025602314728', last_update_time=1758697027208, lifecycle_stage='active', name='EXP_6-Xgboost_hpt', tags={}>

In [6]:
# Step 1: Remap class labels
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove NaN labels
df = df.dropna(subset=['category'])

# Step 3: Train-test split
X_raw = df['clean_comment']
y_raw = df['category']
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# Step 4: TF-IDF fit on training set
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 5: Apply ADASYN only on training set
adasyn = ADASYN(random_state=42)
X_train_vec_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

# Step 6: MLflow logging function
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)
        for key, value in params.items():
            mlflow.log_param(key, value)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")
        return accuracy

# Step 7: Optuna objective for XGBClassifier
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
    }

    model = XGBClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        use_label_encoder=False,
        eval_metric='mlogloss',
        random_state=42
    )

    accuracy = log_mlflow("XGBoost", model, X_train_vec_resampled, X_test_vec, y_train_resampled, y_test, params, trial.number)
    return accuracy

# Step 8: Run Optuna study and log best model & parameters
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgb, n_trials=50)

    best_params = study.best_params
    print("Best parameters:", best_params)
    print("Best accuracy:", study.best_value)

    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        gamma=best_params['gamma'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        use_label_encoder=False,
        eval_metric='mlogloss',
        random_state=42
    )

    # Log best model and best params in MLflow
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", "XGBoost_Best_Model_ADASYN")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", "XGBoost")
        for key, value in best_params.items():
            mlflow.log_param(key, value)

        best_model.fit(X_train_vec_resampled, y_train_resampled)
        mlflow.sklearn.log_model(best_model, "XGBoost_model")

        y_pred = best_model.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

    # Optional visualizations
    optuna.visualization.plot_param_importances(study).show()
    optuna.visualization.plot_optimization_history(study).show()

# Run the experiment
run_optuna_experiment()

[I 2025-09-24 12:27:49,549] A new study created in memory with name: no-name-e12d67d3-28f7-4ed1-82c2-b4a455611f2a
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:27:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/09/24 12:28:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/24 12:29:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_XGBoost_ADASYN_TFIDF_Trigrams at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728/runs/8fabdfe1fb9d4500863a66ff6f04ea7b
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728


[I 2025-09-24 12:29:35,992] Trial 0 finished with value: 0.7785353879721806 and parameters: {'n_estimators': 346, 'max_depth': 3, 'learning_rate': 0.24329833876786336, 'subsample': 0.613227268010546, 'colsample_bytree': 0.9397739997010075, 'gamma': 2.168438690404134, 'reg_alpha': 4.517303648292667, 'reg_lambda': 2.7976156715112843}. Best is trial 0 with value: 0.7785353879721806.
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:29:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/09/24 12:31:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/24 12:31:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_1_XGBoost_ADASYN_TFIDF_Trigrams at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728/runs/04d5371552914a15bfa4600419a8aeeb
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728


[I 2025-09-24 12:31:31,552] Trial 1 finished with value: 0.6830765034774308 and parameters: {'n_estimators': 228, 'max_depth': 6, 'learning_rate': 0.020518450653246374, 'subsample': 0.549071187253046, 'colsample_bytree': 0.6878214724311287, 'gamma': 0.034714867796751814, 'reg_alpha': 3.5557822801400354, 'reg_lambda': 3.9700646291043995}. Best is trial 0 with value: 0.7785353879721806.
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:31:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/09/24 12:33:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/24 12:34:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run Trial_49_XGBoost_ADASYN_TFIDF_Trigrams at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728/runs/f01f7eac330d4644b9a5cb06b9c61a8a
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728


[I 2025-09-24 14:13:08,059] Trial 49 finished with value: 0.7890358652666031 and parameters: {'n_estimators': 458, 'max_depth': 6, 'learning_rate': 0.1677819576643497, 'subsample': 0.9977231520406316, 'colsample_bytree': 0.7957514763833095, 'gamma': 0.7806516227448366, 'reg_alpha': 3.2858421669294247, 'reg_lambda': 3.350616611529648}. Best is trial 43 with value: 0.7940815491613256.


Best parameters: {'n_estimators': 461, 'max_depth': 6, 'learning_rate': 0.2118182334195666, 'subsample': 0.9489600110773789, 'colsample_bytree': 0.7864903456665429, 'gamma': 1.3374019590838637, 'reg_alpha': 0.7880029370689157, 'reg_lambda': 2.871310489936178}
Best accuracy: 0.7940815491613256


c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:13:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/09/24 14:14:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/24 14:14:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost_Best_Model_ADASYN at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728/runs/7131de73d7d043479781d94555b70186
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/754749025602314728
